In [2]:
using ITensors
using Optim
using Random

In [3]:
s = siteinds("S=1/2", 4)
mps = randomMPS(ComplexF64, s; linkdims=4);

In [4]:
struct PState
    pstate::MPS
    label::Int
end

In [5]:
function complex_feature_map(x::Float64)
    s1 = exp(1im * (3π/2) * x) * cospi(0.5 * x)
    s2 = exp(-1im * (2π/2) * x) * sinpi(0.5 * x)
    return [s1, s2]
end

complex_feature_map (generic function with 1 method)

In [6]:
function generate_training_data(samples_per_class::Int)

    class_A_samples = zeros(samples_per_class, 4)
    class_B_samples = ones(samples_per_class, 4)
    all_samples = vcat(class_A_samples, class_B_samples)
    all_labels = Int.(vcat(zeros(size(class_A_samples)[1]), ones(size(class_B_samples)[1])))

    return all_samples, all_labels

end

generate_training_data (generic function with 1 method)

In [7]:
function sample_to_product_state(sample::Vector, site_inds::Vector{Index{Int64}})
    n_sites = length(site_inds)
    product_state = MPS(ComplexF64, site_inds; linkdims=1)
    for j=1:n_sites
        T = ITensor(site_inds[j])
        zero_state, one_state = complex_feature_map(sample[j])
        T[1] = zero_state
        T[2] = one_state
        product_state[j] = T 
    end
    return product_state
end

sample_to_product_state (generic function with 1 method)

In [8]:
function dataset_to_product_state(dataset::Matrix, labels::Vector, sites::Vector{Index{Int64}})

    all_product_states = Vector{PState}(undef, size(dataset)[1])
    for p=1:length(all_product_states)
        sample_pstate = sample_to_product_state(dataset[p, :], sites)
        sample_label = labels[p]
        product_state = PState(sample_pstate, sample_label)
        all_product_states[p] = product_state
    end

    return all_product_states

end

dataset_to_product_state (generic function with 1 method)

In [9]:
all_samples, all_labels = generate_training_data(100)
all_pstates = dataset_to_product_state(all_samples, all_labels, s);

In [79]:
BT = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=724|"S=1/2,Site,n=1") (dim=2|id=104|"S=1/2,Site,n=2") (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [80]:
function loss(B::ITensor)
    product_state = all_pstates[1]
    ps = all_pstates[1].pstate
    lid = 1
    rid = 2
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = norm(yhat[] - y)^2
    loss = 0.5 * diff_sq
    return loss
end

loss (generic function with 2 methods)

In [81]:
loss(BT)

0.0003243776509033321

In [82]:
function gradient(B::ITensor)
    product_state = all_pstates[1]
    ps = all_pstates[1].pstate
    lid = 1
    rid = 2
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return grad

end
    

gradient (generic function with 2 methods)

In [83]:
function flatten_bond_tensor(BT::ITensor)
    """Function to flatten an ITensor so that it can be fed into Optim
    as a vector."""
    # should probably return the indices as well
    # might need checks to ensure correct assignment of indices to values
    flattened_tensor = collect(Iterators.flatten(BT))
    return flattened_tensor, inds(BT)
end

flatten_bond_tensor (generic function with 1 method)

In [84]:
BT_flattened, BT_inds = flatten_bond_tensor(BT);
BT_flattened

16-element Vector{ComplexF64}:
   0.09132171378965329 + 0.017390368794233408im
   -0.3645883853078941 + 0.3605618647117407im
   0.14340821400332981 + 0.05714727193769016im
 -0.061875126932802255 - 0.17839505912915488im
 -0.059128889130315755 + 0.10491041592755732im
  -0.07808140996084341 - 0.03900504662536748im
   -0.2722048578896096 + 0.2337301663445115im
    0.1335115377739747 - 0.03996961257988981im
  0.030787904087829568 + 0.030744209715260335im
   -0.3917483826104733 + 0.0640581484485691im
  -0.25206272469074925 + 0.04194169131606157im
    0.1900386688619175 + 0.07399525904004105im
  0.023333031569183222 - 0.013182212954102091im
   0.12001232965043487 + 0.2898585979223783im
   0.05367282759225719 + 0.15100139065793688im
  -0.31911503057113216 - 0.07530469095936829im

In [85]:
function reconstruct_bond_tensor(BT_flat::Vector, indices)
    BT = ITensor(indices)
    # ORDER OF ASSIGNMENT MUST MATCH THE ORDER OF FLATTENING
    for (n, val) in enumerate(BT_flat)
        BT[n] = val
    end

    return BT

end

reconstruct_bond_tensor (generic function with 2 methods)

In [86]:
BT_reconstructed = reconstruct_bond_tensor(BT_flattened, BT_inds)

ITensor ord=3 (dim=2|id=724|"S=1/2,Site,n=1") (dim=2|id=104|"S=1/2,Site,n=2") (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [87]:
function loss_flat(params::Vector, bt_inds)
    # takes in flattened ITensor
    B = reconstruct_bond_tensor(params, bt_inds)
    product_state = all_pstates[1]
    ps = all_pstates[1].pstate
    lid = 1
    rid = 2
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = norm(yhat[] - y)^2
    loss = 0.5 * diff_sq
    return loss
end

loss_flat (generic function with 1 method)

In [88]:
function gradient_flat(params::Vector, bt_inds)
    B = reconstruct_bond_tensor(params, bt_inds)
    product_state = all_pstates[1]
    ps = all_pstates[1].pstate
    lid = 1
    rid = 2
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return grad

end

gradient_flat (generic function with 1 method)

In [89]:
loss_flat(BT_flattened, BT_inds)

0.0003243776509033321

In [90]:
loss(BT)

0.0003243776509033321

In [91]:
@show gradient_flat(BT_flattened, BT_inds)

gradient_flat(BT_flattened, BT_inds) = ITensor ord=3
Dim 1: (dim=2|id=724|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=104|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 0.0011883168402251339 - 0.0007470713846025681im  -0.0 + 0.0im
                  -0.0 + 0.0im                    -0.0 + 0.0im

[:, :, 2] =
 0.002245664181352213 - 0.002095639237707649im  -0.0 + 0.0im
                 -0.0 + 0.0im                   -0.0 + 0.0im

[:, :, 3] =
 0.003454117495135392 + 0.009944492090671978im  -0.0 + 0.0im
                 -0.0 + 0.0im                   -0.0 + 0.0im

[:, :, 4] =
 0.005514647894042047 - 0.0030900986146534023im  -0.0 + 0.0im
                 -0.0 + 0.0im                    -0.0 + 0.0im


ITensor ord=3 (dim=2|id=724|"S=1/2,Site,n=1") (dim=2|id=104|"S=1/2,Site,n=2") (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [93]:
initial_guess = BT_flattened

16-element Vector{ComplexF64}:
   0.09132171378965329 + 0.017390368794233408im
   -0.3645883853078941 + 0.3605618647117407im
   0.14340821400332981 + 0.05714727193769016im
 -0.061875126932802255 - 0.17839505912915488im
 -0.059128889130315755 + 0.10491041592755732im
  -0.07808140996084341 - 0.03900504662536748im
   -0.2722048578896096 + 0.2337301663445115im
    0.1335115377739747 - 0.03996961257988981im
  0.030787904087829568 + 0.030744209715260335im
   -0.3917483826104733 + 0.0640581484485691im
  -0.25206272469074925 + 0.04194169131606157im
    0.1900386688619175 + 0.07399525904004105im
  0.023333031569183222 - 0.013182212954102091im
   0.12001232965043487 + 0.2898585979223783im
   0.05367282759225719 + 0.15100139065793688im
  -0.31911503057113216 - 0.07530469095936829im

In [94]:
cost = x -> loss_flat(x, BT_inds)

#9 (generic function with 1 method)

In [97]:
grad = x -> gradient_flat(x, BT_inds)

#11 (generic function with 1 method)

In [99]:
@show grad(initial_guess)

grad(initial_guess) = ITensor ord=3
Dim 1: (dim=2|id=724|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=104|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 0.0011883168402251339 - 0.0007470713846025681im  -0.0 + 0.0im
                  -0.0 + 0.0im                    -0.0 + 0.0im

[:, :, 2] =
 0.002245664181352213 - 0.002095639237707649im  -0.0 + 0.0im
                 -0.0 + 0.0im                   -0.0 + 0.0im

[:, :, 3] =
 0.003454117495135392 + 0.009944492090671978im  -0.0 + 0.0im
                 -0.0 + 0.0im                   -0.0 + 0.0im

[:, :, 4] =
 0.005514647894042047 - 0.0030900986146534023im  -0.0 + 0.0im
                 -0.0 + 0.0im                    -0.0 + 0.0im


ITensor ord=3 (dim=2|id=724|"S=1/2,Site,n=1") (dim=2|id=104|"S=1/2,Site,n=2") (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [104]:
x0 = randn(4)+im*randn(4)

4-element Vector{ComplexF64}:
  0.07174118716510826 + 0.8370373962657851im
    1.090178125748597 - 0.7765745558578111im
 -0.12812740300971684 + 0.2986753958333526im
  -1.2644371353177892 + 1.5146559312297998im

In [102]:
optimize(cost, grad, initial_guess,  LBFGS())

MethodError: MethodError: no method matching (::var"#11#12")(::Vector{ComplexF64}, ::Vector{ComplexF64})

Closest candidates are:
  (::var"#11#12")(::Any)
   @ Main ~/Documents/QuantumInspiredML/MPS_MSE/complex-opt/Julia/analytic/optim_complex.ipynb:1


In [106]:
function gradient_flat!(stor, params::Vector, bt_inds)
    B = reconstruct_bond_tensor(params, bt_inds)
    product_state = all_pstates[1]
    ps = all_pstates[1].pstate
    lid = 1
    rid = 2
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    # Assuming `grad` is now a vector that correctly represents the gradient,
    # and `stor` is the storage vector provided by Optim.jl:
    copyto!(stor, grad)
    return nothing  # In-place modification does not need to return the gradient
end

gradient_flat! (generic function with 1 method)

In [107]:
initial_guess = BT_flattened

16-element Vector{ComplexF64}:
   0.09132171378965329 + 0.017390368794233408im
   -0.3645883853078941 + 0.3605618647117407im
   0.14340821400332981 + 0.05714727193769016im
 -0.061875126932802255 - 0.17839505912915488im
 -0.059128889130315755 + 0.10491041592755732im
  -0.07808140996084341 - 0.03900504662536748im
   -0.2722048578896096 + 0.2337301663445115im
    0.1335115377739747 - 0.03996961257988981im
  0.030787904087829568 + 0.030744209715260335im
   -0.3917483826104733 + 0.0640581484485691im
  -0.25206272469074925 + 0.04194169131606157im
    0.1900386688619175 + 0.07399525904004105im
  0.023333031569183222 - 0.013182212954102091im
   0.12001232965043487 + 0.2898585979223783im
   0.05367282759225719 + 0.15100139065793688im
  -0.31911503057113216 - 0.07530469095936829im

In [108]:
cost = x -> loss_flat(x, BT_inds)

#13 (generic function with 1 method)

In [109]:
grad! = (stor, x) -> gradient_flat!(stor, x, BT_inds)

#15 (generic function with 1 method)

In [114]:
@time res = optimize(cost, grad!, initial_guess, LBFGS())

  0.000258 seconds (1.73 k allocations: 268.281 KiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.201534e-36

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 2.11e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.11e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.24e-04 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.70e+32 ≰ 0.0e+00
    |g(x)|                 = 6.41e-19 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    4
    ∇f(x) calls:   4


In [116]:
@time res = optimize(cost, grad!, initial_guess, ConjugateGradient())


  0.000352 seconds (1.89 k allocations: 290.734 KiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.201534e-36

 * Found with
    Algorithm:     Conjugate Gradient

 * Convergence measures
    |x - x'|               = 2.11e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.11e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.24e-04 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.70e+32 ≰ 0.0e+00
    |g(x)|                 = 6.41e-19 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    5
    ∇f(x) calls:   4


In [119]:
result_flattened = Optim.minimizer(res)

16-element Vector{ComplexF64}:
   0.08894508010920302 + 0.018884511563438545im
   -0.3645883853078941 + 0.3605618647117407im
   0.14340821400332981 + 0.05714727193769016im
 -0.061875126932802255 - 0.17839505912915488im
  -0.06362021749302019 + 0.10910169440297261im
  -0.07808140996084341 - 0.03900504662536748im
   -0.2722048578896096 + 0.2337301663445115im
    0.1335115377739747 - 0.03996961257988981im
  0.023879669097558783 + 0.010855225533916376im
   -0.3917483826104733 + 0.0640581484485691im
  -0.25206272469074925 + 0.04194169131606157im
    0.1900386688619175 + 0.07399525904004105im
  0.012303735781099126 - 0.007002015724795285im
   0.12001232965043487 + 0.2898585979223783im
   0.05367282759225719 + 0.15100139065793688im
  -0.31911503057113216 - 0.07530469095936829im

In [122]:
result_as_ITensor = reconstruct_bond_tensor(result_flattened, BT_inds)

ITensor ord=3 (dim=2|id=724|"S=1/2,Site,n=1") (dim=2|id=104|"S=1/2,Site,n=2") (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [123]:
@show result_as_ITensor

result_as_ITensor = ITensor ord=3
Dim 1: (dim=2|id=724|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=104|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 0.08894508010920302 + 0.018884511563438545im    0.14340821400332981 + 0.05714727193769016im
 -0.3645883853078941 + 0.3605618647117407im    -0.061875126932802255 - 0.17839505912915488im

[:, :, 2] =
 -0.06362021749302019 + 0.10910169440297261im  -0.2722048578896096 + 0.2337301663445115im
 -0.07808140996084341 - 0.03900504662536748im   0.1335115377739747 - 0.03996961257988981im

[:, :, 3] =
 0.023879669097558783 + 0.010855225533916376im  -0.25206272469074925 + 0.04194169131606157im
  -0.3917483826104733 + 0.0640581484485691im      0.1900386688619175 + 0.07399525904004105im

[:, :, 4] =
 0.012303735781099126 - 0.007002015724795285im   0.05367282759225719 + 0.15100139065793688im
  0.12001232965043487 + 0.2898585979223783im    -0.31911503057113216 - 0.07530469095936829im


ITensor ord=3 (dim=2|id=724|"S=1/2,Site,n=1") (dim=2|id=104|"S=1/2,Site,n=2") (dim=4|id=850|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}